## Population
Primero creamos la clase poblacion, tendra atributos generales de toda la poblacion como el nombre, tama;o, ploidia y vida media (habran mas seguramente).

Despues se crea la clase individuo, que hereda de poblacion sus atributos y ademas tendra otros tales como el identificador, la edad, el sexo, su genotipo y quiza sus padres **ver como implementar**

### Funciones
#### init
Inicializa los atributos, que son
- **name** :(str) nombre de la poblacion, no relevante
- **size** :(int) tama;o de la poblacion, utilizado para recorrer posteriormente los individuos entre otras
- **ploidy** :(int) indica si la pob esta formada por individuos haploides(1) o diploides(2)
- **vida media** : (int) todavia sin uso, se implementara posteriormente para generar una dist de edades y para hacerla evolucionar.
- **R**: (float [0,0.5]) frecuencia de recombinacion
- **mut**: (N-size tuple [0,1]) frecuencia de mutacion de cada gen 
- **freq**: (N-size dict tuples) frecuencia alelica, se expresa en un diccionario donde la key es el genX y los valores son la frecuencia de su alelo dominante y el recesivo

Estos son los atributos que el usuario debera pasar (tiene valores por defecto), ademas se encuentran otros como:

- **gen1List,gen2List** : (str list) lista con las combinaciones alelicas posibles
- **geneticPool**:(N-size dict tuples) frecuencias genotipicas

#### generateIndividuals()
No se le pasa ningun parametro (posiblemente se cambie ya que el usuario deberia poder elegir ciertas cosas), tampoco devuelve ninguno.
Dentro, genera **indiv** que es una lista de objetos de clase **individuo** (composicion de clases) de tama;o size, ademas printa por pantalla que se ha completado el proceso.

La lista de individuos sera la lista padre original, a partir de la cual se generaran las hijas.

#### printIndiv(show,children)
A esta funcion se le pasan dos parametros, **show**, que permite elegir cuantos individuos se muestran y children, un boobleano que elige si se ense;a la generacion padre o la hija (esto habra que cambiarlo mas adelante)
recorre la lista en un bucle y printa los atributos de cada instancia de individuo en la lista.
#### genotFreq()
Genera las frecuencias genotipicas a partir de las alelicas (pasadas por el usuario o calculadas a partir de los padres), se guardan en un diccionario **geneticPool** que contiene las frecuencias genotipicas para cada gen.
#### getMeanAge()
Calcula la edad media de la poblacion, de momento siempre 0
#### allGenotypes()
Genera una lista para cada gen con sus posibles genotipos (aa o AA..), esta lista se utilizara por Individual. (se podria poner en init pero de momento lo mantengo en funcion separada por claridad)
#### getGenotype()
Crea un diccionario llamado Counter, donde se ira almacenando valores numericos segun aparezcan en el genotipo de los individuos.
- esto habra que cambiarlo para que se muestre 1. en forma de tabla (pandas maybe) 2. muestre la frecuencia de un genotipo completo (gen1 y gen2) 3. calcule frecuencias relativas.

#### evolvePop(gens,k,time)
Esta funcion llevara a cabo el proceso de evolucion, sera el usuario quien la llame, a partir de ese momento y hasta un numero X de generaciones (pasadas por el usuario o predeterminadas) la poblacion comenzara a evolucionar, esto es:
- se dara un proceso de 'mating', donde 2 individuos de la lista de objetos Individual se seleccionan (en concreto sus genotipos **genotype**) y se produce su 'recombinacion'
- su sexo **sex** se escogera de forma aleatoria (en un principio, mas adelante podria verse algun gen de det del sexo gen3=XX o XY)
- la edad **age** se ignorara de momento, 
- tambien podra producirse una mutacion de uno de sus alelos (dominante a recesivo y viceversa), de momento la mutacion no podra crear nuevos alelos (ej. A,a y nuevo c)
- como el tama;o **size** de la poblacion debe mantenerse constante la nueva lista de Individual sera igual que la anterior en tama;o, asi que:
    1. Por cada pareja debe surgir 2 hijos (monogamia) --> sencillo
    *2. Un individuo puede juntarse con mas de un individuo(poligamia simple) --> ligeramente menos sencillo*
    3. Que exista un parametro extra similar a la eficacia biologica, determinado por el genotipo, que afectara al numero de descendientes de un individuo. FALTA desarrollo pero esta seria la forma ideal. --> complejo pero importante
- por ultimo, un parametro **time** que nos diga cada cuantas generaciones queremos guardar informacion sobre la lista de Individuals (para estadisticas y graficos), esto llamaria a otra funcion no definida que iria calculando las estadisticas del individuo que le pasemos en ese momento y las devolveria, este return se almacenaria en una estructura tipo dataframe de tama;o filas: gen//time, col:atributos(sex,genotype,etc) (siendo gen el numero de generaciones que queremos evolucionar la poblacion), que despues se podria representar graficamente.

De momento evolvePop obtiene una nueva lista de individuos generados a partir de la lista self.indiv (lista padre), utilizando el metodo 2 de seleccion de individuos, los individuos creados tendran un self.ide formado por los ide de los padres, esto es util ahora para ver de donde vienen sus genotipos pero mas tarde habra que cambiarlo para que esta informacion se almacene en otro lugar (quiza cada individuo deba tener su propia lista de individuos, mas bien lista de tuplas de n=2 aunque esto no se si haria el proceso mas lento).

Lo siguiente a hacer es:

1. generar una funcion o conjunto de funciones que permitan obtener estadisticas adecuadas (o grafico o tabla pero que resuman bien la informacion de la poblacion)
2. Introducir el parametro mut a la generacion de genotipos
3. ir creando (o sobreescribiendo) las listas ya existentes por nuevos individuos, asi hasta un numero de veces n que sera el numero de generaciones. 

### chooseMate()
Metodo para elegir de la lista de Individuos dos de ellos, este metodo deberia elegir dos y obtener un genotipo resultante que sera el del nuevo individuo. Si no se eliminan las parejas significa que un mismo Individuo puede dar lugar a mas de un hijo.

### findFreqAlleles(ind1,ind2)
Se le pasan como atributos los dos individuos, de los cuales debera obtener las frecuencias alelicas sumando las ocurrencias de A,a y B,b para ambos. Estos valores sobreescribiran el antiguo diccionario freq, esto puede ser un poco confuso ya que antes freq indicaba una caractersitica de la poblacion y ahora indica la de dos individuos, quiza sea mejor crear un nuevo atributo llamado freqInd pero esto hara que sea necesario cambiar ligeramente el metodo genotFreq().


In [118]:
from random import randint
import itertools

#clase poblacion,atributos generales que heredara de los individuos
class Population:
    
    geneticPool = dict()
    
    def __init__(self,size = 100,name="Homo sapiens",ploidy = 2, vida_media=55,
                 R=0.1,mu = (1e-4,1e-4),freq={'A':(0.4,0.6),'B':(0.6,0.4)}):
                 
        self.name = name
        self.size = size
        self.ploidy = ploidy
        self.gen1List = []
        self.gen2List = []
        self.vida_media = vida_media
        self.freq = freq
        self.mu = mu
        self.geneticPool = self.genotFreq()
        self.allGenotypes()
        
    def __str__(self):
        return ''.join([self.name])
    
    #genera indivividuos
    def generateIndividuals(self):
        #ejecuta la funcion para crear las variables globales que contengan los genotipos posibles
        self.indiv = [Individual(i,self.name,
                                 self.size,
                                 self.ploidy,
                                 self.gen1List,
                                 self.gen2List,
                                 self.vida_media,
                                 self.geneticPool) 
                      for i in range(self.size)]
        print("se han generado un total de {} individuos de la poblacion {}"
              .format(self.size,self.name))
        
    #printa individuos        
    def printIndiv(self,show=5,children=True):
        show = abs(show)
        listaAtrib = ['ide','sex','genotype']
        print(*listaAtrib,sep="\t")
        if children==True and hasattr(self,'childrenInd'):
            print("print chidren")
            objectList = self.childrenInd
        else:
            objectList = self.indiv
            
        for x in objectList:
            print (*[getattr(x,y) for y in listaAtrib],sep="\t")
            #contador inverso, si se han enseñado show elementos para la ejecucion
            show += -1
            if show == 0:
                break
                
    #funcion que calcula las frecuencias genotipicas
    def genotFreq(self):
        if self.ploidy == 2:
            for key,lista in self.freq.items():
                self.geneticPool[key] = (lista[0]**2,
                                         lista[0]*lista[1]*2,
                                         lista[1]**2)
                self.geneticPool[key] = [x*self.size for x in self.geneticPool[key]]
        elif self.ploidy == 1:
            self.geneticPool = self.freq
        return self.geneticPool
        
    def getMeanAge(self):
        #obtienes la edad media recorriendo la lista de individuos
        try:
            meanAge = 0
            for obj in self.indiv:
                meanAge += obj.age
            meanAge = round(meanAge/len(self.indiv),2)
            print("la edad media de la poblacion es: ",meanAge)
        except:
            print("No has inicializado la poblacion")
            
    #esto quedara obsoleto       
    def allGenotypes(self):
       #genera una lista de tuplas(n=ploidy=2), cada tupla contiene un alelo
        self.gen1List = [''.join(x) for x in itertools.combinations_with_replacement('Aa',2)]
        self.gen2List = [''.join(x) for x in itertools.combinations_with_replacement('Bb',2)]
                
    #muestra informacion genotipo
    def getGenotype(self):
        counter = {'AA':0,'Aa':0,'aa':0,'BB':0,'Bb':0,'bb':0}      
        for individuo in self.indiv:
            for key in counter:
                if individuo.genotype['gene_1'] == key or individuo.genotype['gene_2']==key:
                    counter[key] += 1
        print(counter)
        
    def evolvePop(self):
        #crea lista vacia de individuos
        self.childrenInd = []
        #va introduciendo nuevos individuos hasta llegar al size de la poblacion
        for x in range(self.size):
            self.childrenInd.append(self.chooseMate())
        
        
    def chooseMate(self):
        #elige dos individuos de forma aleatoria
        ind1,ind2 = random.choices(self.indiv,k=2)
        print(f"Individuo 1: {ind1}\n Individuo 2: {ind2}")
        #genera las frecuencias alelicas de ambos individuos
        self.findFreqAlleles(ind1,ind2)
        #genera las frecuencias genotipicas
        self.genotFreq()
        #nuevo nombre que se le pasara al Individual
        Ind_Name = ind1.ide+ind2.ide
        #genera un nuevo individuo y lo devuelve al metodo evolvePop
        return Individual(Ind_Name,
                         self.name,
                         self.size,
                         self.ploidy,
                         self.gen1List,
                         self.gen2List,
                         self.vida_media,
                         self.geneticPool)
    
    def findFreqAlleles(self,ind1,ind2):
        self.freq = dict()
        #debe haber alguna forma de hacerlo menos 'hard coded'
        sizeA = len(ind1.genotype['gene_1'])+len(ind2.genotype['gene_1'])
        freq_A = (ind1.genotype['gene_1'].count('A')+ind2.genotype['gene_1'].count('A'))/sizeA
        freq_a = ind1.genotype['gene_1'].count('a')+ind1.genotype['gene_1'].count('a')/sizeA
        self.freq['A']=(freq_A,freq_a)
        
        sizeB = len(ind1.genotype['gene_2'])+len(ind2.genotype['gene_2'])
        freq_B = (ind1.genotype['gene_2'].count('B')+ind2.genotype['gene_2'].count('B'))/sizeB
        freq_b = (ind1.genotype['gene_2'].count('b')+ind2.genotype['gene_2'].count('b'))/sizeB
        self.freq['B']=(freq_B,freq_b)
    


# Individual
Individual se trata de una clase que se crea de forma indirecta, llamando un metodo de Population, esta clase no es accesible al usuario. 
Esta clase consta de los atributos y metodos necesarios para dar lugar a un individuo que formara parte de la poblacion, los atributos son: 

## atributos
- Atributos pasados de la clase Population +...
- Sex: tipo string male and female
- Ide: identificador, para cada individuo (mirar de construir la lista de individuos con forma de diccionario para acceder por su identificador)
- Age: numeric int, comienza en 0
- genotype: quiza un diccionario tipo {gen1:'AA',gen2:'Bb'}, en cualquier caso se puede hacer un conjunto con todos los posibles genotipos y escoger aleatoriamente.

## Metodos

### init
Inicializa los atributos vistos anteriormente, la forma de inicializarlos puede no ser la correcta ya que en muchos se esta llamando a un metodo de la clase pero de momento funciona

### sex
Le da un valor de sexo a cada individuo de forma aleatoria al 50%, no requiere ningun parametro, devuelve male o female
### edad
Al igual que sex, asigna una edad a cada individuo. Esto habra que repensarlo porque la edad comienza en 0 para todos, dependiendo de como lo quiera implementar cuando la poblacion empiece a evolucionar esta edad debera ir aumentando para todos, algunos individuos iran muriendo = eliminando de la lista de objetos en la poblacion. Si este metodo hace correr el tiempo quiza seria mejor tenerlo en poblacion ya que el tiempo corre para todos igual *revisar*
### firstGenotipo y chooseGenotype
Genera un diccionario gen:alelos para 2 genes en total (mirar de aumentarlo mas adelante), los alelos se obtienen de la lista de alelos posibles de la clase population y se asigna uno de estos aleatoriamente si asi se quiere, si no se puede inicializar a todos los individuos con el mismo genotipo Aa/Bb. *ver como el usuario pueda pasar esta forma de generar genotipo*, seguramente desde generate individuals se deba pasar un parametro


In [119]:
#clase individuo, ya no hereda de poblacion
import itertools
import random

class Individual():

    def __init__(self,nom,name,size,ploidy,gen1List,gen2List,
                 vida_media,geneticPool):
        
        #quiza el "name"no sea necesario para nada
        self.spName = name
        self.spSize = size
        self.spPloidy = ploidy
        self.gene_1_List = gen1List
        self.gene_2_List = gen2List
        self.vida_media = vida_media
        self.geneticPool = geneticPool
        
        self.sex = self.sex()
        self.ide = "id-"+str(nom)
        self.age = 0
        self.genotype = self.firstGenotipo() #or self.generateGen()
        
    def sex(self):
        if randint(0,1)==0:
            return "Male"
        else:
            return 'Female'
        
    def edad(self):
        #el usuario debera pasar algun parametro para indicar la dist por edades
        pass
    
    def firstGenotipo(self,homogeneous=False):
        if homogeneous:
            return {'gen1':'Aa','gen2':'Bb'}
        else:
            return self.chooseGenotype()
        
    def chooseGenotype(self):
        dict = {}

        dict['gene_1']= ''.join(random.choices(self.gene_1_List,weights=self.geneticPool['A'],k=1))
        dict['gene_2']= ''.join(random.choices(self.gene_2_List,weights=self.geneticPool['B'],k=1))
        
        return dict
    
#en un principio esto no es necesario ya que no interesa conocer a un individuo concreto
    def __str__(self):
        return ("este individuo es {}, su sexo es {} y su genotipo es {}"
              .format(self.ide,self.sex,self.genotype))
    
    def childrenGen(self,list_a,list_b):
        pass


### Notas:

- Se ha eliminado la estructura de herencia, ahora un metodo (generateIndividuals) de la clase Population crea las nuevas instancias Individual, se les pasan los atributos de Population.
- Seguir o no el modelo de Wright-Fisher para evolucionar a la poblacion, si queremos que sean generaciones separadas hara mas facil su implementacion y no sera necesaria una edad para los individuos ya que todos viven y mueren en una gen. Sin embargo hacer generaciones solapantes amplia el abanico de posibilidades. 
 

In [120]:
#se crea una nueva poblacion donde se especifican caracteristicas generales de esta
    #size es el numero de individuos
    #name el nombre de la especie
    #ploidy es la ploidia de la poblacion (haploide=1,diploide=2)
    #vida media es la vida media
    #freq son las frecuencias alelicas en cada locus, es una tupla de diccionarios
    #mu es la tasa de mutacion de los alelos (de A a a y viceversa..)
    
shark = Population(size=100,
                     name="Megadolon",
                     ploidy=2,
                     vida_media=23,
                     freq={'A':(0.4,0.6),'B':(0.6,0.4)})

In [121]:
#se generan individuos en esa poblacion
shark.generateIndividuals()

se han generado un total de 100 individuos de la poblacion Megadolon


In [122]:
#parametro opcional show, permite elegir cuantos elementos se muestran (por defecto se muestran 10)
shark.printIndiv(show=10)

ide	sex	genotype
id-0	Male	{'gene_1': 'Aa', 'gene_2': 'BB'}
id-1	Male	{'gene_1': 'Aa', 'gene_2': 'BB'}
id-2	Male	{'gene_1': 'aa', 'gene_2': 'Bb'}
id-3	Male	{'gene_1': 'aa', 'gene_2': 'Bb'}
id-4	Male	{'gene_1': 'aa', 'gene_2': 'Bb'}
id-5	Male	{'gene_1': 'AA', 'gene_2': 'Bb'}
id-6	Male	{'gene_1': 'AA', 'gene_2': 'Bb'}
id-7	Male	{'gene_1': 'aa', 'gene_2': 'Bb'}
id-8	Female	{'gene_1': 'aa', 'gene_2': 'bb'}
id-9	Male	{'gene_1': 'Aa', 'gene_2': 'BB'}


In [123]:
shark.getMeanAge()

la edad media de la poblacion es:  0.0


In [124]:
#devuelve un diccionario con un resumen de las frecuencias absolutas para cada gen
shark.getGenotype()

{'AA': 17, 'Aa': 46, 'aa': 37, 'BB': 29, 'Bb': 53, 'bb': 18}


In [125]:
shark.chooseMate()

Individuo 1: este individuo es id-55, su sexo es Female y su genotipo es {'gene_1': 'Aa', 'gene_2': 'BB'}
 Individuo 2: este individuo es id-26, su sexo es Male y su genotipo es {'gene_1': 'Aa', 'gene_2': 'BB'}


In [126]:
shark.evolvePop()

Individuo 1: este individuo es id-62, su sexo es Male y su genotipo es {'gene_1': 'aa', 'gene_2': 'bb'}
 Individuo 2: este individuo es id-30, su sexo es Female y su genotipo es {'gene_1': 'aa', 'gene_2': 'BB'}
Individuo 1: este individuo es id-64, su sexo es Male y su genotipo es {'gene_1': 'Aa', 'gene_2': 'bb'}
 Individuo 2: este individuo es id-86, su sexo es Female y su genotipo es {'gene_1': 'Aa', 'gene_2': 'BB'}
Individuo 1: este individuo es id-35, su sexo es Male y su genotipo es {'gene_1': 'aa', 'gene_2': 'Bb'}
 Individuo 2: este individuo es id-43, su sexo es Female y su genotipo es {'gene_1': 'Aa', 'gene_2': 'Bb'}
Individuo 1: este individuo es id-71, su sexo es Female y su genotipo es {'gene_1': 'aa', 'gene_2': 'bb'}
 Individuo 2: este individuo es id-51, su sexo es Female y su genotipo es {'gene_1': 'aa', 'gene_2': 'Bb'}
Individuo 1: este individuo es id-26, su sexo es Male y su genotipo es {'gene_1': 'Aa', 'gene_2': 'BB'}
 Individuo 2: este individuo es id-68, su sexo es 

In [127]:
shark.printIndiv(children=True)

ide	sex	genotype
print chidren
id-id-62id-30	Female	{'gene_1': 'aa', 'gene_2': 'Bb'}
id-id-64id-86	Male	{'gene_1': 'Aa', 'gene_2': 'Bb'}
id-id-35id-43	Male	{'gene_1': 'aa', 'gene_2': 'BB'}
id-id-71id-51	Female	{'gene_1': 'aa', 'gene_2': 'Bb'}
id-id-26id-68	Female	{'gene_1': 'Aa', 'gene_2': 'BB'}


In [ ]:
shark.getGenotype()

In [ ]:
#comienza a correr el tiempo (no implementado)
#humanos.age()

In [ ]:
#check if population attribute gen1List is actually a object class parameter
print(hasattr(Population,'summary'))
print(hasattr(Population,'Population.size'))
print(hasattr(Individual,'vida_media'))
print(vars(Population))

In [116]:
shark.checkAttr()

True
